<a href="https://colab.research.google.com/github/GrainSack/neural-tangents/blob/main/MFwithNTK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Neural_CF-master

/content/drive/MyDrive/Neural_CF-master


In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 75% !important; }</style>"))

import pandas as pd
import numpy as np

In [ ]:
#from google.colab import files
#uploaded = files.upload()

In [6]:
file_path = '/content/drive/MyDrive/Neural_CF-master/lastfm-dataset-360K'
df = pd.read_csv(file_path + '/usersha1-artmbid-artname-plays.tsv', delimiter='\t', header=None)

df = df.drop(df.columns[2], axis=1)
df.columns = ['user', 'item', 'plays']
df = df.dropna()
df = df.loc[df.plays != 0]

In [7]:
df.shape

(17309518, 3)

In [8]:
df.head()

,user,item,plays
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,706


In [9]:
print('user 수:', len(np.unique(list(df['user'])))) 
print('artist 수:', len(np.unique(list(df['item']))))

user 수: 358858
artist 수: 160112


In [10]:
def prepare_analy_dataset(df):
    """
    데이터 로드 함수 
    
    uids: train user
    iids: train item
    users: 전체 user          
    items: 전체 item
    df_train: train data
    df_test: test data
    """
    # user 10000명 샘플링 
    unique_user_lst = list(np.unique(df['user'])) #358857명 
    sample_user_idx = np.random.choice(len(unique_user_lst), 10000, replace=False)
    sample_user_lst = [unique_user_lst[idx] for idx in sample_user_idx]
    
    df = df[df['user'].isin(sample_user_lst)]
    df = df.reset_index(drop=True)

    # 1명 이상의 artist데이터가 있는 user만 사용 
    df_count = df.groupby(['user']).count()
    df['count'] = df.groupby('user')['user'].transform('count')
    df = df[df['count'] > 1]

    # user, item 아이디 부여 
    df['user_id'] = df['user'].astype("category").cat.codes
    df['item_id'] = df['item'].astype("category").cat.codes

    # lookup 테이블 생성 
    item_lookup = df[['item_id', 'item']].drop_duplicates()
    item_lookup['item_id'] = item_lookup.item_id.astype(str)

    # train, test 데이터 생성 
    df = df[['user_id', 'item_id', 'plays']] 
    df_train, df_test = train_test_split(df)

    # 전체 user, item 리스트 생성 
    users = list(np.sort(df.user_id.unique())) 
    items = list(np.sort(df.item_id.unique())) 

    # train user, item 리스트 생성 
    rows = df_train['user_id'].astype(int)   
    cols = df_train['item_id'].astype(int)
    values = list(df_train.plays) 
    
    uids = np.array(rows.tolist())
    iids = np.array(cols.tolist())

    # 각 user마다 negative item 생성 
    df_neg = get_negatives(uids, iids, items, df_test)

    return uids, iids, df_train, df_test, df_neg, users, items, item_lookup

def get_negatives(uids, iids, items, df_test):
    """
    negative item 리스트 생성함수
    """
    negativeList = []
    test_u = df_test['user_id'].values.tolist() 
    test_i = df_test['item_id'].values.tolist() 
 
    test_ratings = list(zip(test_u, test_i)) # test (user, item)세트 
    zipped = set(zip(uids, iids))            # train (user, item)세트

    for (u, i) in test_ratings:
        
        negatives = []
        negatives.append((u, i))
        for t in range(100):
            j = np.random.randint(len(items))     # neg_item j 1개 샘플링 
            while (u, j) in zipped:               # j가 train에 있으면 다시뽑고, 없으면 선택 
                j = np.random.randint(len(items)) 
            negatives.append(j)
        negativeList.append(negatives) # [(0,pos), neg, neg, ...]

    df_neg = pd.DataFrame(negativeList)

    return df_neg

def mask_first(x):

    result = np.ones_like(x) 
    result[0] = 0  # [0,1,1,....]
    
    return result

def train_test_split(df):
    """
    train, test 나누는 함수
    """
    df_test = df.copy(deep=True)
    df_train = df.copy(deep=True)
    
    # df_test
    # user_id와 holdout_item_id(user가 플레이한 아이템 중 1개)뽑기 
    df_test = df_test.groupby(['user_id']).first() 
    df_test['user_id'] = df_test.index
    df_test = df_test[['user_id', 'item_id', 'plays']]
    df_test = df_test.reset_index(drop=True)
    
    # df_train 
    # user_id 리스트에 make_first()적용 
    mask = df.groupby(['user_id'])['user_id'].transform(mask_first).astype(bool)
    df_train = df.loc[mask]  

    return df_train, df_test

def get_train_instances(uids, iids, num_neg, num_items):
    """
    모델에 사용할 train 데이터 생성 함수 
    """
    user_input, item_input, labels = [],[],[]
    zipped = set(zip(uids, iids)) # train (user, item) 세트

    for (u, i) in zip(uids, iids):
        
        # pos item 추가 
        user_input.append(u) #[u]
        item_input.append(i) #[pos_i]
        labels.append(1)     #[1]

        # neg item 추가 
        for t in range(num_neg):
            
            j = np.random.randint(num_items)     # neg_item j num_neg(4)개 샘플링
            while (u, j) in zipped:              # u가 j를 이미 선택했다면 
                j = np.random.randint(num_items) # 다시 샘플링 
                
            user_input.append(u) # [u1, u1,  u1,  ...]
            item_input.append(j) # [pos_i, neg_j1, neg_j2, ...]
            labels.append(0)     # [1, 0,  0,  ...]

    return user_input, item_input, labels

In [11]:
uids, iids, df_train, df_test, df_neg, users, items, item_lookup = prepare_analy_dataset(df)

In [12]:
df_train.head(10)

,user_id,item_id,plays
1,0,32117,1540
2,0,7898,1403
3,0,15735,1308
4,0,4388,1214
5,0,5478,1197
6,0,3822,945
7,0,44460,895
8,0,47358,887
9,0,3912,833
10,0,7939,721


In [13]:
df_train.shape

(472097, 3)

In [14]:
user_input, item_input, labels = get_train_instances(uids, iids, num_neg=4, num_items=len(items))

In [15]:
print('df_train의 첫번째 행: (user_id, item_id)=', (uids[0], iids[0])) 
print('df_train의 두번째 행: (user_id, item_id)=', (uids[1], iids[1])) 

df_train의 첫번째 행: (user_id, item_id)= (0, 32117)
df_train의 두번째 행: (user_id, item_id)= (0, 7898)


In [16]:
for i, (user_id, item_id, label) in enumerate(zip(user_input[0:10], item_input[0:10], labels[0:10])):
    if i==0 or i==5:
        print('(user_id, postive_item_id, label):', (user_id, item_id, label))
    else:
        print('(user_id, negative_item_id, label):', (user_id, item_id, label))

(user_id, postive_item_id, label): (0, 32117, 1)
(user_id, negative_item_id, label): (0, 16826, 0)
(user_id, negative_item_id, label): (0, 38790, 0)
(user_id, negative_item_id, label): (0, 40056, 0)
(user_id, negative_item_id, label): (0, 36382, 0)
(user_id, postive_item_id, label): (0, 7898, 1)
(user_id, negative_item_id, label): (0, 4910, 0)
(user_id, negative_item_id, label): (0, 28710, 0)
(user_id, negative_item_id, label): (0, 30522, 0)
(user_id, negative_item_id, label): (0, 36932, 0)


In [17]:
df_test.head(10)

,user_id,item_id,plays
0,0,40780,2618
1,1,15453,744
2,2,37624,157
3,3,33130,802
4,4,25257,341
5,5,21499,835
6,6,1227,1046
7,7,11773,4418
8,8,2215,168
9,9,30763,1776


In [18]:
df_test.shape

(9997, 3)

In [19]:
df_neg.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,"(0, 40780)",18447,14159,44985,43226,12988,38499,40761,18261,14179,...,24176,25296,421,37182,47583,11273,17766,10523,9190,41012
1,"(1, 15453)",37389,38644,6809,10012,45516,12896,3300,1927,30358,...,6337,38598,1264,24409,47353,44961,1350,136,36001,38596
2,"(2, 37624)",32649,5050,34405,48404,43756,19581,1985,36912,28457,...,39536,7744,13616,11921,1140,24986,8178,19577,37338,18657
3,"(3, 33130)",7706,24412,18241,38745,43345,38608,43129,44668,2578,...,14984,36433,10382,13264,42512,37398,28594,16568,23680,1532
4,"(4, 25257)",24924,9387,36992,5516,30135,10238,40738,44509,5677,...,4496,26082,2747,15780,23746,8108,31261,20875,368,47955


In [20]:
df_neg.shape

(9997, 101)

In [2]:
!sudo apt-get install -y --no-install-recommends libnvinfer6=6.0.1-1+cuda10.1 \
    libnvinfer-dev=6.0.1-1+cuda10.1 \
    libnvinfer-plugin6=6.0.1-1+cuda10.1

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package libnvinfer6
E: Version '6.0.1-1+cuda10.1' for 'libnvinfer-dev' was not found
E: Unable to locate package libnvinfer-plugin6


In [4]:
!export TF_CPP_MIN_LOG_LEVEL="2"

In [ ]:
# %pip install neural-tangents
# !pip install -r /content/drive/MyDrive/infinite_ac_cf_main/requirements.txt
# !pip install sciPy

In [ ]:
!pip install neural-tangents

In [9]:
!pip install jax==0.3.0
#!pip install jax jaxlib==0.3.0
#!pip install jax tf2jax==0.3.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached jax-0.3.0-py3-none-any.whl
  Attempting uninstall: jax
    Found existing installation: jax 0.4.4
    Uninstalling jax-0.4.4:
      Successfully uninstalled jax-0.4.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf2jax 0.3.3 requires jax>=0.3.14, but you have jax 0.3.0 which is incompatible.
neural-tangents 0.6.2 requires jax>=0.4.3, but you have jax 0.3.0 which is incompatible.


In [6]:
pip list | grep jax

jax                           0.4.4
jaxlib                        0.3.0
tf2jax                        0.3.3


In [4]:
%cd /content/drive/MyDrive/infinite_ac_cf_main

/content/drive/MyDrive/infinite_ac_cf_main


In [5]:
pwd

'/content/drive/MyDrive/infinite_ac_cf_main'

In [11]:
import neural_tangents

ModuleNotFoundError: ignored

Import

In [7]:
import jax
import numpy as np
import functools
import h5py, sys, os
import copy
import h5py
import gc
import pickle
import jax.numpy as jnp
import time
import random
import heapq
from sklearn.utils import shuffle
from jax import scipy as sp
from jax import numpy as jnp
from neural_tangents import stax
from collections import defaultdict
from scipy.sparse import csr_matrix
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
os.environ["TF_FORCE_UNIFIED_MEMORY"] = "1"
os.environ["XLA_PYTHON_CLIENT_ALLOCATOR"] = "platform"

ImportError: ignored

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

Utils

In [16]:
def get_common_path(hyper_params):
    ret = "{}_users_{}_depth_{}_".format(
        hyper_params['dataset'], hyper_params['user_support'],
        hyper_params['depth']
    )
    
    if hyper_params['grid_search_lamda']: ret += "grid_search_lamda_"
    else: ret += "lamda_{}_".format(hyper_params['lamda'])
    
    ret += "seed_{}".format(hyper_params['seed'])
    return ret

def get_item_count_map(data):
    item_count = defaultdict(int)
    for u, i, r in data.data['train']: item_count[i] += 1
    return item_count

def get_item_propensity(hyper_params, data, A = 0.55, B = 1.5):
    item_freq_map = get_item_count_map(data)
    item_freq = [ item_freq_map[i] for i in range(hyper_params['num_items']) ]
    num_instances = hyper_params['num_interactions']

    C = (np.log(num_instances)-1)*np.power(B+1, A)
    wts = 1.0 + C*np.power(np.array(item_freq)+B, -A)
    return np.ravel(wts)

def file_write(log_file, s, dont_print=False):
    if dont_print == False: print(s)
    if log_file is None: return
    f = open(log_file, 'a')
    f.write(s+'\n')
    f.close()

def log_end_epoch(hyper_params, metrics, step, time_elpased, metrics_on = '(TEST)', dont_print = False):
    string2 = ""
    for m in metrics: string2 += " | " + m + ' = ' + str("{:2.4f}".format(metrics[m]))
    string2 += ' ' + metrics_on

    ss  = '| end of step {:4d} | time = {:5.2f}'.format(step, time_elpased)
    ss += string2
    file_write(hyper_params['log_file'], ss, dont_print = dont_print)

Data

In [17]:
with open('test.pickle', 'rb') as f:
   test_pickle = pickle.load(f)

class Dataset:
    def __init__(self, hyper_params):
        self.data = load_raw_dataset(hyper_params['dataset'])
        self.set_of_active_users = list(set(self.data['train'][:, 0].tolist()))            
        self.hyper_params = self.update_hyper_params(hyper_params)

    def update_hyper_params(self, hyper_params):
        updated_params = copy.deepcopy(hyper_params)
        
        self.num_users, self.num_items = self.data['num_users'], self.data['num_items']
        self.num_interactions = self.data['num_interactions']

        # Update hyper-params to have some basic data stats
        updated_params.update({
            'num_users': self.num_users,
            'num_items': self.num_items,
            'num_interactions': self.num_interactions
        })

        return updated_params

    def sample_users(self, num_to_sample):
        if num_to_sample == -1: 
            ret = self.data['train_matrix']
        else: 
            sampled_users = np.random.choice(self.set_of_active_users, num_to_sample, replace=False)
            sampled_interactions = self.data['train'][np.in1d(self.data['train'][:, 0], sampled_users)]
            ret = csr_matrix(
                ( np.ones(sampled_interactions.shape[0]), (sampled_interactions[:, 0], sampled_interactions[:, 1]) ),
                shape = (self.num_users, self.num_items)
            )

        # This just removes the users which were not sampled
        return jnp.array(ret[ret.getnnz(1)>0].todense())

def load_raw_dataset(dataset, data_path = None, index_path = None):
    if data_path is None or index_path is None:
        data_path, index_path = [
            "/content/drive/MyDrive/ml-1m{}/total_data.hdf5".format(dataset),
            "/content/drive/MyDrive/ml-1m{}/index.npz".format(dataset)
        ]

    with h5py.File(data_path, 'r') as f: data = np.array(list(zip(f['user'][:], f['item'][:], f['rating'][:])))
    index = np.array(np.load(index_path)['data'], dtype = np.int32)

    def remap(data, index):
        ## Counting number of unique users/items before
        valid_users, valid_items = set(), set()
        for at, (u, i, r) in enumerate(data):
            if index[at] != -1:
                valid_users.add(u)
                valid_items.add(i)

        ## Map creation done!
        user_map = dict(zip(list(valid_users), list(range(len(valid_users)))))
        item_map = dict(zip(list(valid_items), list(range(len(valid_items)))))

        return user_map, item_map

    user_map, item_map = remap(data, index)

    new_data, new_index = [], []
    for at, (u, i, r) in enumerate(data):
        if index[at] == -1: continue
        new_data.append([ user_map[u], item_map[i], r ])
        new_index.append(index[at])
    data = np.array(new_data, dtype = np.int32)
    index = np.array(new_index, dtype = np.int32)

    def select(data, index, index_val):
        final = data[np.where(index == index_val)[0]]
        final[:, 2] = 1.0
        return final.astype(np.int32)

    ret = {
        'item_map': item_map,
        'train':  select(data, index, 0),
        'val': select(data, index, 1),
        #'test': test_pickle
        'test': select(data, index, 2)
    }
    #print(ret['test'])

    num_users = int(max(data[:, 0]) + 1)
    num_items = len(item_map)
    print(num_users, num_items)

    del data, index ; gc.collect()

    def make_user_history(arr):
        ret = [ set() for _ in range(num_users) ] #[ set() for _ in range(num_users) ]
        for u, i, r in arr:
            if i >= num_items: continue
            ret[int(u)].add(int(i))
        return ret

    ret['train_positive_set'] = make_user_history(ret['train'])
    ret['val_positive_set'] = make_user_history(ret['val'])
    ret['test_positive_set'] = make_user_history(ret['test'])

    ret['train_matrix'] = csr_matrix(
        ( np.ones(ret['train'].shape[0]), (ret['train'][:, 0].astype(np.int32), ret['train'][:, 1].astype(np.int32)) ),
        shape = (num_users, num_items)
    )

    ret['val_matrix'] = csr_matrix(
        ( np.ones(ret['val'].shape[0]), (ret['val'][:, 0].astype(np.int32), ret['val'][:, 1].astype(np.int32)) ),
        shape = (num_users, num_items)
    )

    # Negatives will be used for AUC computation
    ret['negatives'] = [ set() for _ in range(num_users) ]
    for u in range(num_users):
        while len(ret['negatives'][u]) < 50:
            rand_item = np.random.randint(0, num_items)
            if rand_item in ret['train_positive_set'][u]: continue
            if rand_item in ret['test_positive_set'][u]: continue
            ret['negatives'][u].add(rand_item)
        ret['negatives'][u] = list(ret['negatives'][u])
    ret['negatives'] = np.array(ret['negatives'], dtype=np.int32)

    ret.update({
        'num_users': num_users,
        'num_items': num_items,
        'num_interactions': len(ret['train']),
    })
    print("# users:", num_users)
    print("# items:", num_items)
    print("# interactions:", len(ret['train']))

    return ret

Preprocess

In [18]:
BASE_PATH = '/content/drive/MyDrive/ml-1m'
#base_path

def prep_movielens(ratings_file_path):
    f = open(ratings_file_path, "r")
    users, items, ratings = [], [], []

    line = f.readline()
    while line:
        u, i, r,_ = line.strip().split("::")
        users.append(int(u))
        items.append(int(i))
        ratings.append(float(r))
        line = f.readline()

    min_user = min(users)
    num_users = len(set(users))

    data = [ [] for _ in range(num_users) ]
    for i in range(len(users)):
        data[users[i] - min_user].append([ items[i], ratings[i] ])

    return rating_data(data)

class rating_data:
    def __init__(self, data):
        self.data = data

        self.index = [] # 0: train, 1: validation, 2: test, -1: removed due to user frequency < 3
        for user_data in self.data:
            for _ in range(len(user_data)): self.index.append(42)

    def train_test_split(self):
        at = 0

        for user in range(len(self.data)):
            first_split_point = int(0.8 * len(self.data[user]))
            second_split_point = int(0.9 * len(self.data[user]))

            indices = np.arange(len(self.data[user]))
            np.random.shuffle(indices)

            for timestep, (item, rating) in enumerate(self.data[user]):
                if len(self.data[user]) < 3: self.index[at] = -1
                else:
                    # Force atleast one element in user history to be in test
                    if timestep == indices[0]: self.index[at] = 2
                    else:
                        if timestep in indices[:first_split_point]: self.index[at] = 0
                        elif timestep in indices[first_split_point:second_split_point]: self.index[at] = 1
                        else: self.index[at] = 2
                at += 1

        assert at == len(self.index)
        self.complete_data_stats = None

    def save_index(self, path):
        os.makedirs(path, exist_ok = True)
        with open(path + "/index.npz", "wb") as f: np.savez_compressed(f, data = self.index)

    def save_data(self, path):
        flat_data = []
        for u in range(len(self.data)):
            flat_data += list(map(lambda x: [ u ] + x, self.data[u]))
        flat_data = np.array(flat_data)

        shape = [ len(flat_data) ]

        os.makedirs(path, exist_ok = True)
        with h5py.File(path + '/total_data.hdf5', 'w') as file:
            dset = {}
            dset['user'] = file.create_dataset("user", shape, dtype = 'i4', maxshape = shape, compression="gzip")
            dset['item'] = file.create_dataset("item", shape, dtype = 'i4', maxshape = shape, compression="gzip")
            dset['rating'] = file.create_dataset("rating", shape, dtype = 'f', maxshape = shape, compression="gzip")

            dset['user'][:] = flat_data[:, 0]
            dset['item'][:] = flat_data[:, 1]
            dset['rating'][:] = flat_data[:, 2]

if __name__ == "__main__":
    if len(sys.argv) < 2: 
        print("This file needs the dataset name as the first argument...")
        exit(0)
    
    dataset = sys.argv[1]

    print("\n\n!!!!!!!! STARTED PROCESSING {} !!!!!!!!".format(dataset))

    #if dataset in [ 'ml-1m' ]: total_data = prep_movielens(BASE_PATH +"/ratings.dat") #"/ratings.dat"
    total_data = prep_movielens(BASE_PATH +"/ratings.dat")

    total_data.save_data(BASE_PATH + "{}/".format(dataset))
    total_data.train_test_split()
    total_data.save_index(BASE_PATH + "{}/".format(dataset))



!!!!!!!! STARTED PROCESSING -f !!!!!!!!


Hyper_params

In [19]:
hyper_params = {
	'dataset': 'ml-1m', 
	'float64': False,

	'depth': 1,
	'grid_search_lamda': True,
	'lamda': 1, # Only used if grid_search_lamda == False

	# Number of users to keep (randomly)
	'user_support': -1, # -1 implies use all users
	'seed': 42,
}


Model

In [21]:
def make_kernelized_rr_forward(hyper_params):
    _, _, kernel_fn = FullyConnectedNetwork(
        depth=hyper_params['depth'],
        num_classes=hyper_params['num_items']
    )
    # NOTE: Un-comment this if the dataset size is very big (didn't need it for experiments in the paper)
    # kernel_fn = nt.batch(kernel_fn, batch_size=128)
    kernel_fn = functools.partial(kernel_fn, get='ntk')

    @jax.jit
    def kernelized_rr_forward(X_train, X_predict, reg=0.1):
        K_train = kernel_fn(X_train, X_train)
        K_predict = kernel_fn(X_predict, X_train)
        K_reg = (K_train + jnp.abs(reg) * jnp.trace(K_train) * jnp.eye(K_train.shape[0]) / K_train.shape[0])     
        return jnp.dot(K_predict, sp.linalg.solve(K_reg, X_train, sym_pos=True))

    return kernelized_rr_forward, kernel_fn

def FullyConnectedNetwork( 
    depth,
    W_std = 2 ** 0.5, 
    b_std = 0.1,
    num_classes = 10,
    parameterization = 'ntk'
):
    activation_fn = stax.Relu()
    dense = functools.partial(stax.Dense, W_std=W_std, b_std=b_std, parameterization=parameterization)

    layers = [stax.Flatten()]
    # NOTE: setting width = 1024 doesn't matter as the NTK parameterization will stretch this till \infty
    for _ in range(depth): layers += [dense(1024), activation_fn] 
    layers += [stax.Dense(num_classes, W_std=W_std, b_std=b_std, parameterization=parameterization)]

    return stax.serial(*layers)


Evaluate

In [22]:
#import jax
#import numpy as np
#import jax.numpy as jnp
#from numba import jit, float64

INF = float(1e6)

def evaluate(hyper_params, kernelized_rr_forward, data, item_propensity, train_x, topk = [ 10, 100 ], test_set_eval = False):
    preds, y_binary, metrics = [], [], {}
    for kind in [ 'HR', 'NDCG', 'PSP' ]:
        for k in topk: 
            metrics['{}@{}'.format(kind, k)] = 0.0

    # Train positive set -- these items will be set to -infinity while prediction on the val/test set
    train_positive_list = list(map(list, data.data['train_positive_set']))
    if test_set_eval:
        for u in range(len(train_positive_list)): train_positive_list[u] += list(data.data['val_positive_set'][u])

    # Train positive interactions (in matrix form) as context for prediction on val/test set
    eval_context = data.data['train_matrix']
    if test_set_eval: eval_context += data.data['val_matrix']

    # What needs to be predicted
    to_predict = data.data['val_positive_set']
    if test_set_eval: to_predict = data.data['test_positive_set']

    bsz = 20_000 # These many users
    for i in range(0, hyper_params['num_users'], bsz):
        temp_preds = kernelized_rr_forward(train_x, eval_context[i:i+bsz].todense(), reg = hyper_params['lamda'])
        #np.save('/content/drive/MyDrive/infinite_ac_cf_main/kernel', temp_preds) # dual parameter save
        metrics, temp_preds, temp_y = evaluate_batch(
            data.data['negatives'][i:i+bsz], np.array(temp_preds), 
            train_positive_list[i:i+bsz], to_predict[i:i+bsz], item_propensity, 
            topk, metrics
        )
        
        preds += temp_preds
        y_binary += temp_y

    y_binary, preds = np.array(y_binary), np.array(preds)
    if (True not in np.isnan(y_binary)) and (True not in np.isnan(preds)):
        metrics['AUC'] = round(fast_auc(y_binary, preds), 4)

    for kind in [ 'HR', 'NDCG', 'PSP' ]:
        for k in topk: 
            metrics['{}@{}'.format(kind, k)] = round(
                float(100.0 * metrics['{}@{}'.format(kind, k)]) / hyper_params['num_users'], 4
            )

    metrics['num_users'] = int(train_x.shape[0])
    metrics['num_interactions'] = int(jnp.count_nonzero(train_x.astype(np.int8)))

    return metrics

def evaluate_batch(auc_negatives, logits, train_positive, test_positive_set, item_propensity, topk, metrics, train_metrics = False):
    # AUC Stuff
    temp_preds, temp_y = [], []
    for b in range(len(logits)):
        temp_preds += np.take(logits[b], np.array(list(test_positive_set[b]))).tolist()
        temp_y += [ 1.0 for _ in range(len(test_positive_set[b])) ]

        temp_preds += np.take(logits[b], auc_negatives[b]).tolist()
        temp_y += [ 0.0 for _ in range(len(auc_negatives[b])) ]
    # Marking train-set consumed items as negative INF
    for b in range(len(logits)): logits[b][ train_positive[b] ] = -INF

    indices = (-logits).argsort()[:, :max(topk)].tolist()

    for k in topk: 
        for b in range(len(logits)):
            num_pos = float(len(test_positive_set[b]))

            metrics['HR@{}'.format(k)] += float(len(set(indices[b][:k]) & test_positive_set[b])) / float(min(num_pos, k))

            test_positive_sorted_psp = sorted([ item_propensity[x] for x in test_positive_set[b] ])[::-1]

            dcg, idcg, psp, max_psp = 0.0, 0.0, 0.0, 0.0
            for at, pred in enumerate(indices[b][:k]):
                if pred in test_positive_set[b]: 
                    dcg += 1.0 / np.log2(at + 2)
                    psp += float(item_propensity[pred]) / float(min(num_pos, k))
                if at < num_pos: 
                    idcg += 1.0 / np.log2(at + 2)
                    max_psp += test_positive_sorted_psp[at]

            metrics['NDCG@{}'.format(k)] += dcg / idcg
            metrics['PSP@{}'.format(k)] += psp / max_psp

    return metrics, temp_preds, temp_y

#@jit(float64(float64[:], float64[:]))
def fast_auc(y_true, y_prob):
    y_true = y_true[np.argsort(y_prob)]
    nfalse, auc = 0, 0
    for i in range(len(y_true)):
        nfalse += (1 - y_true[i])
        auc += y_true[i] * nfalse
    return auc / (nfalse * (len(y_true) - nfalse))

Main model

In [59]:
def train(hyper_params, data):

    # This just instantiates the function
    kernelized_rr_forward, kernel_fn = make_kernelized_rr_forward(hyper_params)
    #np.save('/content/drive/MyDrive/infinite_ac_cf_main/kernelized_rr_forward', kernelized_rr_forward) # x_save.npy
    sampled_matrix = data.sample_users(hyper_params['user_support']) # Random user sample

    '''
    NOTE: No training required! We will compute dual-variables \alpha on the fly in `kernelized_rr_forward`
          However, if we needed to perform evaluation multiple times, we could pre-compute \alpha like so:
    
    import jax, jax.numpy as jnp, jax.scipy as sp
    @jax.jit
    def precompute_alpha(X, lamda=0.1):
        K = kernel_fn(X, X)
        K_reg = (K + jnp.abs(lamda) * jnp.trace(K) * jnp.eye(K.shape[0]) / K.shape[0])
        return sp.linalg.solve(K_reg, X, sym_pos=True)
    alpha = precompute_alpha(sampled_matrix, lamda=0.1) # Change for the desired value of lamda
    '''

    # Used for computing the PSP-metric
    item_propensity = get_item_propensity(hyper_params, data)
    
    # Evaluation
    start_time = time.time()

    VAL_METRIC = "HR@100"
    best_metric, best_lamda = None, None

    # Validate on the validation-set
    for lamda in [ 0.0, 1.0, 5.0, 20.0, 50.0, 100.0 ] if hyper_params['grid_search_lamda'] else [ hyper_params['lamda'] ]:
        hyper_params['lamda'] = lamda
        val_metrics = evaluate(hyper_params, kernelized_rr_forward, data, item_propensity, sampled_matrix)
        if (best_metric is None) or (val_metrics[VAL_METRIC] > best_metric): best_metric, best_lamda = val_metrics[VAL_METRIC], lamda

    # Return metrics with the best lamda on the test-set
    hyper_params['lamda'] = best_lamda
    test_metrics = evaluate(hyper_params, kernelized_rr_forward, data, item_propensity, sampled_matrix, test_set_eval = True)
    
    log_end_epoch(hyper_params, test_metrics, 0, time.time() - start_time)
    start_time = time.time()

    return test_metrics, val_metrics

def main(hyper_params, gpu_id = None):
    if gpu_id is not None: os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)
    if 'float64' in hyper_params and hyper_params['float64'] == True: config.update('jax_enable_x64', True)

    np.random.seed(hyper_params['seed'])
    random.seed(hyper_params['seed'])

    os.makedirs("./results/logs/", exist_ok=True)
    hyper_params['log_file'] = "./results/logs/" + get_common_path(hyper_params) + ".txt"
    
    data = Dataset(hyper_params)
    hyper_params = copy.deepcopy(data.hyper_params) # Updated w/ data-stats

    return train(hyper_params, data)

if __name__ == "__main__":
    test, val = main(hyper_params)

6040 3706
# users: 6040
# items: 3706
# interactions: 791718
| end of step    0 | time = 209.60 | HR@10 = 31.8885 | HR@100 = 60.5264 | NDCG@10 = 33.2327 | NDCG@100 = 42.9653 | PSP@10 = 3.2817 | PSP@100 = 6.6276 | AUC = 0.9456 | num_users = 6040.0000 | num_interactions = 791718.0000 (TEST)


In [71]:
list(test)

{'HR@10': 31.8885,
 'HR@100': 60.5264,
 'NDCG@10': 33.2327,
 'NDCG@100': 42.9653,
 'PSP@10': 3.2817,
 'PSP@100': 6.6276,
 'AUC': 0.9456,
 'num_users': 6040,
 'num_interactions': 791718}

In [72]:
list(val)

{'HR@10': 16.1385,
 'HR@100': 42.0721,
 'NDCG@10': 16.4508,
 'NDCG@100': 25.1162,
 'PSP@10': 1.8948,
 'PSP@100': 6.1856,
 'AUC': 0.9305,
 'num_users': 6040,
 'num_interactions': 791718}

In [30]:
data = Dataset(hyper_params)

6040 3706
# users: 6040
# items: 3706
# interactions: 791718


In [42]:
data.data['train_matrix'].size

791718

In [44]:
import pandas as pd

In [57]:
list(data.data)

['item_map',
 'train',
 'val',
 'test',
 'train_positive_set',
 'val_positive_set',
 'test_positive_set',
 'train_matrix',
 'val_matrix',
 'negatives',
 'num_users',
 'num_items',
 'num_interactions']

Neural MF

Data loder

In [73]:
class Loader():

    def __init__(self):
        pass

    def load_dataset(self):
        """
        데이터 로드 함수

        uids: train user
        iids: train item
        users: 전체 user
        items: 전체 item
        df_train
        df_test
        """
        # 데이터 로드
        file_path = '/content/drive/MyDrive/Neural_CF-master/lastfm-dataset-360K'
        df = pd.read_csv(file_path + '/usersha1-artmbid-artname-plays.tsv', delimiter='\t', header=None)
        df = df.drop(df.columns[2], axis=1)
        df.columns = ['user', 'item', 'plays']
        df = df.dropna()
        df = df.loc[df.plays != 0]

        # user 샘플링
        sample_num = 100000
        unique_user_lst = list(np.unique(df['user']))  # 358857명
        sample_user_idx = np.random.choice(len(unique_user_lst), sample_num, replace=False)
        sample_user_lst = [unique_user_lst[idx] for idx in sample_user_idx]
        df = df[df['user'].isin(sample_user_lst)]
        df = df.reset_index(drop=True)

        # 1명 이상의 artist 데이터가 있는 user 만 사용
        df_count = df.groupby(['user']).count()
        df['count'] = df.groupby('user')['user'].transform('count')
        df = df[df['count'] > 1]

        # user, item 아이디 부여
        df['user_id'] = df['user'].astype("category").cat.codes
        df['item_id'] = df['item'].astype("category").cat.codes

        # lookup 테이블 생성
        item_lookup = df[['item_id', 'item']].drop_duplicates()
        item_lookup['item_id'] = item_lookup.item_id.astype(str)

        # train, test 데이터 생성
        df = df[['user_id', 'item_id', 'plays']]
        df_train, df_test = self.train_test_split(df)

        # 전체 user, item 리스트 생성
        users = list(np.sort(df.user_id.unique()))
        items = list(np.sort(df.item_id.unique()))

        # train user, item 리스트 생성
        rows = df_train['user_id'].astype(int)
        cols = df_train['item_id'].astype(int)
        values = list(df_train.plays)

        uids = np.array(rows.tolist())
        iids = np.array(cols.tolist())

        # 각 user 마다 negative item 생성
        df_neg = self.get_negatives(uids, iids, items, df_test)

        return uids, iids, df_train, df_test, df_neg, users, items, item_lookup

    def get_negatives(self, uids, iids, items, df_test):
        """
        negative item 리스트 생성함수
        """
        negativeList = []
        test_u = df_test['user_id'].values.tolist()
        test_i = df_test['item_id'].values.tolist()

        test_ratings = list(zip(test_u, test_i))  # test (user, item)세트
        zipped = set(zip(uids, iids))             # train (user, item)세트

        for (u, i) in test_ratings:

            negatives = []
            negatives.append((u, i))
            for t in range(100):
                j = np.random.randint(len(items))     # neg_item j 1개 샘플링
                while (u, j) in zipped:               # j가 train에 있으면 다시뽑고, 없으면 선택
                    j = np.random.randint(len(items))
                negatives.append(j)
            negativeList.append(negatives) # [(0,pos), neg, neg, ...]

        df_neg = pd.DataFrame(negativeList)

        return df_neg

    def mask_first(self, x):

        result = np.ones_like(x)
        result[0] = 0  # [0,1,1,....]

        return result

    def train_test_split(self, df):
        """
        train, test 나누는 함수
        """
        df_test = df.copy(deep=True)
        df_train = df.copy(deep=True)

        # df_test
        # user_id와 holdout_item_id(user가 플레이한 아이템 중 1개)뽑기
        df_test = df_test.groupby(['user_id']).first()
        df_test['user_id'] = df_test.index
        df_test = df_test[['user_id', 'item_id', 'plays']]
        df_test = df_test.reset_index(drop=True)

        # df_train
        # user_id 리스트에 make_first()적용
        mask = df.groupby(['user_id'])['user_id'].transform(self.mask_first).astype(bool)
        df_train = df.loc[mask]

        return df_train, df_test

    def get_train_instances(self, uids, iids, num_neg, num_items):
        """
        모델에 사용할 train 데이터 생성 함수
        """
        user_input, item_input, labels = [],[],[]
        zipped = set(zip(uids, iids)) # train (user, item) 세트

        for (u, i) in zip(uids, iids):

            # pos item 추가
            user_input.append(u)  # [u]
            item_input.append(i)  # [pos_i]
            labels.append(1)      # [1]

            # neg item 추가
            for t in range(num_neg):

                j = np.random.randint(num_items)      # neg_item j num_neg 개 샘플링
                while (u, j) in zipped:               # u가 j를 이미 선택했다면
                    j = np.random.randint(num_items)  # 다시 샘플링

                user_input.append(u)  # [u1, u1,  u1,  ...]
                item_input.append(j)  # [pos_i, neg_j1, neg_j2, ...]
                labels.append(0)      # [1, 0,  0,  ...]

        return user_input, item_input, labels

Metric

In [74]:
class Metric:

    def __init__(self):
        pass

    def get_hits(self, k_ranked, holdout):
        """
        hit 생성 함수
        hit := holdout(df_test의 item)이 K순위 내에 있는지 여부
        """
        for item in k_ranked:
            if item == holdout:
                return 1
        return 0

    def eval_rating(self, idx, test_ratings, test_negatives, K, model):
        """
        holdout(df_test의 item)이 K순위 내에 있는지 평가하는 함수
        """
        items = test_negatives[idx]      # negative items [neg_item_id, ... ] (1,100)
        user_idx = test_ratings[idx][0]  # [user_id, item_id][0]
        holdout = test_ratings[idx][1]   # [user_id, item_id][1]
        items.append(holdout)            # holdout 추가 [neg_item_id, ..., holdout] (1,101)

        # prediction
        predict_user = np.full(len(items), user_idx, dtype='int32').reshape(-1, 1)  # [[user_id], ...], (101, 1)
        np_items = np.array(items).reshape(-1, 1)                                   # [[item_id], ... ], (101, 1)

        predictions = model.predict([predict_user, np_items])
        predictions = predictions.flatten().tolist()
        item_to_pre_score = {item:pre for item, pre in zip(items, predictions)}

        # 점수가 높은 상위 k개 아이템 리스트
        k_ranked = heapq.nlargest(K, item_to_pre_score, key=item_to_pre_score.get)

        # holdout이 상위 K 순위에 포함 되는지 체크
        # {1:포함, 0:포함x}
        hits = self.get_hits(k_ranked, holdout)

        return hits

    def evaluate_top_k(self, df_neg, df_test, model, K=10):
        """
        TOP-K metric을 사용해 모델을 평가하는 함수
        """
        hits = []
        test_u = df_test['user_id'].values.tolist()
        test_i = df_test['item_id'].values.tolist()

        test_ratings = list(zip(test_u, test_i))
        df_neg = df_neg.drop(df_neg.columns[0], axis=1)
        test_negatives = df_neg.values.tolist()  # [[(user_id, item_id=holdout)], neg_item,... ] (1,100)

        # user 샘플링
        sample_idx_lst = np.random.choice(len(test_ratings), int(len(test_ratings) * 0.3))
        for user_idx in sample_idx_lst:  # 전체 사용: range(len(test_ratings))

            hitrate = self.eval_rating(user_idx, test_ratings, test_negatives, K, model)
            hits.append(hitrate)  # ex. [1,0,1,1,0,...] (1, df_test.shape[0])

        return hits


GMF

In [75]:
class GMP:

    def __init__(self, user_num, item_num):

        latent_features = 8

        # User embedding
        user = Input(shape=(1,), dtype='int32')
        user_embedding = Embedding(user_num, latent_features, input_length=user.shape[1])(user)
        user_embedding = Flatten()(user_embedding)

        # Item embedding
        item = Input(shape=(1,), dtype='int32')
        item_embedding = Embedding(item_num, latent_features, input_length=item.shape[1])(item)
        item_embedding = Flatten()(item_embedding)

        # Merge
        concatenated = Multiply()([user_embedding, item_embedding])

        # Output
        output_layer = Dense(1, kernel_initializer='lecun_uniform', name='output_layer')(concatenated) # 1,1 / h(8,1)초기화

        # Model
        self.model = Model([user, item], output_layer)
        self.model.compile(optimizer='adam', loss='binary_crossentropy')

    def get_model(self):
        model = self.model
        return model

MLP

In [76]:
class MLP:

    def __init__(self, user_num, item_num):

        # User embedding
        user = Input(shape=(1,), dtype='int32')
        user_embedding = Embedding(user_num, 32, input_length=user.shape[1])(user)
        user_embedding = Flatten()(user_embedding)

        # Item embedding
        item = Input(shape=(1,), dtype='int32')
        item_embedding = Embedding(item_num, 32, input_length=item.shape[1])(item)
        item_embedding = Flatten()(item_embedding)

        # Merge
        concatenated = Concatenate()([user_embedding, item_embedding])
        dropout = Dropout(rate=0.2)(concatenated)

        # Layer1
        layer_1 = Dense(units=64, activation='relu', name='layer1')(dropout)  # (64,1)
        dropout1 = Dropout(rate=0.2, name='dropout1')(layer_1)                # (64,1)
        batch_norm1 = BatchNormalization(name='batch_norm1')(dropout1)        # (64,1)

        # Layer2
        layer_2 = Dense(units=32, activation='relu', name='layer2')(batch_norm1)  # (32,1)
        dropout2 = Dropout(rate=0.2, name='dropout2')(layer_2)                    # (32,1)
        batch_norm2 = BatchNormalization(name='batch_norm2')(dropout2)            # (32,1)

        # Layer3
        layer_3 = Dense(units=16, activation='relu', name='layer3')(batch_norm2)  # (16,1)

        # Layer4
        layer_4 = Dense(units=8, activation='relu', name='layer4')(layer_3)  # (8,1)

        # Output
        output_layer = Dense(1, kernel_initializer='lecun_uniform', name='output_layer')(layer_4)  # (1,1) / h(8,1)초기화

        # Model
        self.model = Model([user, item], output_layer)
        self.model.compile(optimizer='adam', loss='binary_crossentropy')

    def get_model(self):
        model = self.model
        return model

NeuralMF

In [77]:
class NeuMF:

    def __init__(self, user_num, item_num):

        latent_features = 8

        # Input
        user = Input(shape=(1,), dtype='int32')
        item = Input(shape=(1,), dtype='int32')

        # User embedding for GMF
        gmf_user_embedding = Embedding(user_num, latent_features, input_length=user.shape[1])(user)
        gmf_user_embedding = Flatten()(gmf_user_embedding)

        # Item embedding for GMF
        gmf_item_embedding = Embedding(item_num, latent_features, input_length=item.shape[1])(item)
        gmf_item_embedding = Flatten()(gmf_item_embedding)

        # User embedding for MLP
        mlp_user_embedding = Embedding(user_num, 32, input_length=user.shape[1])(user)
        mlp_user_embedding = Flatten()(mlp_user_embedding)

        # Item embedding for MLP
        mlp_item_embedding = Embedding(item_num, 32, input_length=item.shape[1])(item)
        mlp_item_embedding = Flatten()(mlp_item_embedding)

        # GMF layers
        gmf_mul =  Multiply()([gmf_user_embedding, gmf_item_embedding])

        # MLP layers
        mlp_concat = Concatenate()([mlp_user_embedding, mlp_item_embedding])
        mlp_dropout = Dropout(0.2)(mlp_concat)

        # Layer1
        mlp_layer_1 = Dense(units=64, activation='relu', name='mlp_layer1')(mlp_dropout)  # (64,1)
        mlp_dropout1 = Dropout(rate=0.2, name='dropout1')(mlp_layer_1)                    # (64,1)
        mlp_batch_norm1 = BatchNormalization(name='batch_norm1')(mlp_dropout1)            # (64,1)

        # Layer2
        mlp_layer_2 = Dense(units=32, activation='relu', name='mlp_layer2')(mlp_batch_norm1)  # (32,1)
        mlp_dropout2 = Dropout(rate=0.2, name='dropout2')(mlp_layer_2)                        # (32,1)
        mlp_batch_norm2 = BatchNormalization(name='batch_norm2')(mlp_dropout2)                # (32,1)

        # Layer3
        mlp_layer_3 = Dense(units=16, activation='relu', name='mlp_layer3')(mlp_batch_norm2)  # (16,1)

        # Layer4
        mlp_layer_4 = Dense(units=8, activation='relu', name='mlp_layer4')(mlp_layer_3)       # (8,1)

        # merge GMF + MLP
        merged_vector = tf.keras.layers.concatenate([gmf_mul, mlp_layer_4])
        
        # Output layer
        output_layer = Dense(1, kernel_initializer='lecun_uniform', name='output_layer')(merged_vector) # 1,1 / h(8,1)초기화

        # Model
        self.model = Model([user, item], output_layer)
        self.model.compile(optimizer= 'adam', loss= 'binary_crossentropy')

    def get_model(self):
        model = self.model
        return model


In [ ]:
class Run:

    def __init__(self):

        # data 로드
        loader = Loader()

        print('start data load..')

        num_neg = 4
        uids, iids, self.df_train, self.df_test, \
        self.df_neg, self.users, self.items, item_lookup = loader.load_dataset()
        user_input, item_input, labels = loader.get_train_instances(uids, iids, num_neg, len(self.items))

        print('end data load..')

        # input data 준비
        user_data_shuff, item_data_shuff, label_data_shuff = shuffle(user_input, item_input, labels)
        self.user_data_shuff = np.array(user_data_shuff).reshape(-1,1)
        self.item_data_shuff = np.array(item_data_shuff).reshape(-1,1)
        self.label_data_shuff = np.array(label_data_shuff).reshape(-1,1)

    def run(self):

        nmf = GMP(len(self.users), len(self.items))  # Neural Collaborative Filtering
        self.model = nmf.get_model()
        self.model.fit([self.user_data_shuff, self.item_data_shuff], self.label_data_shuff, epochs=20,
                       batch_size=256, verbose=1)

        return self.model

    def calculate_top_k_metric(self):
        metric = Metric()
        hit_lst = metric.evaluate_top_k(self.df_neg, self.df_test, self.model, K=10)
        hit = np.mean(hit_lst)

        return hit

if __name__ == '__main__':

    ncf = Run()
    model = ncf.run()

    # top-k metric
    top_k_metric = ncf.calculate_top_k_metric()
    print('metric:', top_k_metric)

    # user 한 명에 대한 prediction 예시
    user_id = 0
    user_candidate_item = np.array([134, 6783, 27888, 8362, 25]).reshape(-1, 1)
    user_input = np.full(len(user_candidate_item), user_id, dtype='int32').reshape(-1, 1)

    predictions = model.predict([user_input, user_candidate_item])
    predictions = predictions.flatten().tolist()
    item_to_pre_score = {item[0]: pre for item, pre in zip(user_candidate_item, predictions)}  # 후보 아이템 별 예측값
    item_to_pre_score = dict(sorted(item_to_pre_score.items(), key=lambda x: x[1], reverse=True))

    recommend_item_lst = list(item_to_pre_score.keys())
    print('recommend:', recommend_item_lst)

start data load..
end data load..
Epoch 1/20
31327/92194 [=========>....................] - ETA: 22:44 - loss: 0.4442